# Bayesian Hierarchical Model (BHM) 

Parameter estimation written for CRDM task as part of the IDM dataset collected online with Mturk

Extended to work for all datasets

### Import modules, libraries, etc

In [11]:
# Built-in/Generic Imports
import os,sys
import glob,time

# Libs :: all are part of idm_env, except for arviz and pymc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import pymc as pm
import arviz as az

# This reduces the amount of pymc output, can comment or change level to see more information
import logging
logger = logging.getLogger("pymc")
logger.setLevel(logging.ERROR)

# explicitly use a path to let the script know where the IDM_model is located to import model_functions
parent = '/Users/pizarror/IDM'
# adding the parent directory to the sys.path.
sys.path.append(parent)
from IDM_model.src import model_functions as mf


### Diagnostic plots

Run `diagnistic_plots()` individually by subject :: trace, posterior, bivariate densities, rank plots

In [12]:
def diganostic_plots(trace,experiment='experiment',utility_dir='/tmp/',subject='23_IDM_0001',task='crdm',coords={},var_names=['alpha','beta','gamma'],figsize=(10,10)):

    bh_dir = os.path.join(utility_dir,subject,task,'bh')
    if not os.path.exists(bh_dir):
        os.makedirs(bh_dir)
    print('Saving diagnostic plots to bh_dir : {}'.format(bh_dir))

    title_dict = {'fontsize':15}

    # 2by2 : rows 2 varialbes, cols 2 for distribution and sampled values
    axes = az.plot_trace(trace, var_names=var_names,coords=coords,compact=False)
    for r in range(axes.shape[0]):
        for c in range(axes.shape[1]):
            axes[r,c].set_title('{}: {}'.format(subject,var_names[r]))
    plt.tight_layout()
    fig_fn = os.path.join(bh_dir,'{}_{}_trace_plot.{}.eps'.format(subject,task,experiment))
    plt.savefig(fig_fn,format='eps')
    plt.close()
    
    axes = az.plot_pair(trace,kind='kde', coords=coords,var_names=var_names,marginals=True)
    axes[0,0].set_title(subject,fontdict=title_dict)
    axes[1,0].set_ylabel(var_names[1])
    axes[1,0].set_xlabel(var_names[0])
    plt.tight_layout()
    fig_fn = os.path.join(bh_dir,'{}_{}_bivariate_densities.{}.eps'.format(subject,task,experiment))
    plt.savefig(fig_fn,format='eps')
    plt.close()
    
    axes = az.plot_posterior(trace,var_names=var_names,coords=coords)
    # print(axes.shape)
    for c in range(axes.shape[0]):
        axes[c].set_title('{}: {}'.format(subject,var_names[c]),fontdict=title_dict)
    fig_fn = os.path.join(bh_dir,'{}_{}_posterior.{}.eps'.format(subject,task,experiment))
    plt.savefig(fig_fn,format='eps')
    plt.close()

    fig, axes = plt.subplots(1,len(var_names), figsize=figsize)
    az.plot_rank(trace,var_names=var_names,coords=coords,ax=axes)
    for c in range(axes.shape[0]):
        axes[c].set_title('{}: {}'.format(subject,var_names[c]),fontdict=title_dict)
    fig.tight_layout()
    fig_fn = os.path.join(bh_dir,'{}_{}_rank_plot_bars.{}.eps'.format(subject,task,experiment))
    plt.savefig(fig_fn,format='eps')
    plt.close()

    fig, axes = plt.subplots(1,len(var_names), figsize=figsize)
    axes = az.plot_rank(trace,var_names=var_names, coords=coords,kind="vlines",vlines_kwargs={'lw':0}, marker_vlines_kwargs={'lw':3},ax=axes)
    for c in range(axes.shape[0]):
        axes[c].set_title('{}: {}'.format(subject,var_names[c]),fontdict=title_dict)
    fig.tight_layout()
    fig_fn = os.path.join(bh_dir,'{}_{}_rank_plot_lines.{}.eps'.format(subject,task,experiment))
    plt.savefig(fig_fn,format='eps')
    plt.close()



### Load data

We will load the data from all participants so we can run the modeling schema

In [13]:
def read_load_data(subject='23_IDM_0144',fn='/tmp',domain='gain',cols=[]):
    crdm_df = pd.read_csv(fn)
    task='crdm'
    if ('loss' in domain) or ('combined' in domain):
        if 'loss' not in crdm_df['crdm_domain'].dropna().unique():
            return []

    if ('loss' in domain) or ('gain' in domain):
        # only select by domain when gain or loss
        crdm_df = mf.get_by_domain(crdm_df,domain=domain,task=task,verbose=False)
    crdm_df,response_rate = mf.drop_non_responses(crdm_df,task=task,verbose=False) 
    crdm_df = mf.drop_pract(crdm_df,task=task)
    data = mf.get_data(crdm_df,cols,task='crdm')[0]
    data['subject'] = subject
    return data

def dirs_and_data(root_dir='/tmp',dataset='IDM',domain='gain'):

    dataset_dir = os.path.join(root_dir,dataset)
    split_dir = os.path.join(dataset_dir,'split')
    utility_dir = os.path.join(dataset_dir,'utility')
    save_dir = os.path.join(utility_dir,'BHM/crdm')
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Takes about 10 seconds for 149 subjects form Mturk data
    task = 'crdm'
    subjs = sorted(glob.glob(os.path.join(split_dir,'*')))
    cols = ['crdm_choice','crdm_sure_p','crdm_lott_p','crdm_sure_amt','crdm_lott_amt','crdm_amb_lev']
    data = pd.DataFrame(columns=['subject']+cols)
    subj_id = 0
    for s in subjs:
        subject = os.path.basename(s)
        fn  = os.path.join(s,task,'{}_{}.csv'.format(os.path.basename(s),task))
        if os.path.exists(fn):
            subj_data = read_load_data(subject=subject,fn=fn,domain=domain,cols=cols)
            if subj_data.empty:
                continue
            for c in cols:
                subj_data[c] = subj_data[c].astype(float)        
            subj_data['subject_id'] = int(subj_id)
            subj_id = subj_id+1
            data = pd.concat([data,subj_data],ignore_index=True)

    return utility_dir,save_dir,data

#  amplitude (+1, -1) parameter to handle gain and loss
def get_amplitude(amt):
    return np.divide(amt,np.abs(amt))

def extract_cols(data):
    subjects = data['subject'].unique()
    # nb_subj = subjects.shape[0]
    # nb_trials = data.shape[0]//nb_subj
    subj_id_list = data['subject_id'].to_list()
    subj_id = [int(s) for s in subj_id_list]
    # old_id = np.array([ [s]*nb_trials for s in range(nb_subj) ]).flatten()

    sure_p = data['crdm_sure_p'].values
    lott_p = data['crdm_lott_p'].values
    sure_amt = data['crdm_sure_amt'].values
    lott_amt = data['crdm_lott_amt'].values
    ambig = data['crdm_amb_lev'].values
    choices = data['crdm_choice'].values

    # amplitude parameter to handle gain and loss
    sure_amp = get_amplitude(sure_amt)
    lott_amp = get_amplitude(lott_amt)

    return subjects,subj_id,sure_p,lott_p,sure_amt,sure_amp,lott_amt,lott_amp,ambig,choices


### Bayesian Hierarchical Model

Choice of prior distribution and parameters used to define were developed under `parameter_receovery.ipynb`


In [14]:

# This is the meat of the script that is used to estimate the parameters of the BHM 

def run_BHM(save_dir,experiment,domain,data):
    
    subjects,subj_id,sure_p,lott_p,sure_amt,sure_amp,lott_amt,lott_amp,ambig,choices = extract_cols(data)

    tStep1 = time.time()

    # We will fit a model for each subject
    with pm.Model() as model_simple:

        # Hyperparameters for alpha, beta
        mu_alpha_hyper = pm.Normal('mu_alpha_hyper',mu=-0.635,sigma=0.05)
        sd_alpha_hyper = pm.Normal('sd_alpha_hyper',mu=0.418,sigma=0.01)

        mu_beta_hyper = pm.Normal('mu_beta_hyper',mu=0.254,sigma=0.05)
        sd_beta_hyper = pm.Normal('sd_beta_hyper',mu=0.575,sigma=0.01)
        
        # mu_gamma_hyper = pm.Normal('mu_gamma_hyper',mu=3.011,sigma=0.05)
        sd_gamma_hyper = pm.Normal('sd_gamma_hyper',mu=2.046,sigma=0.01)

        alpha = pm.LogNormal('alpha',mu=mu_alpha_hyper, sigma=sd_alpha_hyper,shape=np.size(np.unique(subj_id)))
        beta = pm.Normal('beta',mu=mu_beta_hyper,sigma=sd_beta_hyper,shape=np.size(np.unique(subj_id)))
        gamma = pm.HalfNormal('gamma',sigma=sd_gamma_hyper,shape=np.size(np.unique(subj_id)))
        
        prob = pm.Deterministic('prob', 1 / (1 + pm.math.exp(-gamma[subj_id] * 
                                                                (lott_amp*(np.abs(lott_amt)**alpha[subj_id])*(lott_p-beta[subj_id]*ambig/2)
                                                                - sure_amp*(np.abs(sure_amt)**alpha[subj_id])*sure_p)))) 
        
        y_1 = pm.Bernoulli('y_1',p=prob,observed=choices)

        # trace_prior = pm.sample(100, tune=20, cores=2,target_accept=0.95,progressbar=True)
        trace_prior = pm.sample(10000, tune=1000, cores=5,target_accept=0.99,progressbar=True)


    summary= az.summary(trace_prior,round_to=10)
    fn = os.path.join(save_dir,'BHM_model_summary_{}_{}.csv'.format(experiment,domain))
    print('Saving to : {}'.format(fn))
    summary.to_csv(fn)

    fn = os.path.join(save_dir,'BHM_model_trace_{}_{}.pkl'.format(experiment,domain))
    print('Saving to : {}'.format(fn))
    with open(fn,'wb') as buff:
        pickle.dump({'trace':trace_prior},buff)

    print('Time to complete {} aggregate BHM : {} minutes'.format(len(subjects),(time.time() - tStep1)/60.0))
    return trace_prior,subjects,subj_id


# Extract parameter estimates and save
def extract_mean(save_dir='',var_names=['alpha','beta','gamma'],subjects=[],experiment='v002',domain='gain'):
    fn = os.path.join(save_dir,'BHM_model_summary_{}_{}.csv'.format(experiment,domain))
    nb_subjects = len(subjects)
    df = pd.read_csv(fn,index_col=0)
    df_bhm = pd.DataFrame([],columns=var_names)
    df_bhm['subject'] = subjects
    df_bhm['domain'] = domain
    for var in var_names:
        ind_list = ['{}[{}]'.format(var,sub_id) for sub_id in range(nb_subjects)]
        df_bhm[var] = df.loc[df.index.isin(ind_list)]['mean'].reset_index(drop=True)

    # save dataframe to file    
    split_CRDM_fn = os.path.join(os.path.dirname(save_dir),'split_CRDM_BHM_{}.csv'.format(domain))
    print('Saving split BHM : {}'.format(split_CRDM_fn))
    df_bhm.to_csv(split_CRDM_fn)    
    display(df_bhm)
    
    return df_bhm

In [15]:
# Set the paths needed to find the data and load

# The user of the script can edit root_dir and dataset variables to get it to work for their dataset.
# This script will work given the data is stored in the appropriate BIDS format in the split directory
root_dir = '/Users/pizarror/Library/CloudStorage/OneDrive-SharedLibraries-NationalInstitutesofHealth/NIMH CDN lab - Datasets'
# root_dir = '/Volumes/UCDN/datasets/'
dataset = 'SDAN'

# log the BHM experiment version, in case we cahnge it in the future
experiment='v002'

for domain in ['gain','loss','combined']:
    utility_dir,save_dir,data = dirs_and_data(root_dir=root_dir,dataset=dataset,domain=domain)
    if data.empty:
        # catch for when CRDM data by domain is not present
        continue
    trace_prior,subjects,subj_id = run_BHM(save_dir,experiment,domain,data)
    df_bhm = extract_mean(save_dir=save_dir,var_names=['alpha','beta','gamma'],
                          subjects=subjects,experiment=experiment,domain=domain)    
    for s in set(subj_id):
        coords={'alpha_dim_0': [s],'beta_dim_0': [s],'gamma_dim_0':[s]}
        diganostic_plots(trace_prior,experiment=experiment,utility_dir=utility_dir,subject=subjects[s],
                         coords=coords,var_names=['alpha','beta','gamma'],figsize=(10,10))

    

Saving to : /Users/pizarror/Library/CloudStorage/OneDrive-SharedLibraries-NationalInstitutesofHealth/NIMH CDN lab - Datasets/SDAN/utility/BHM/crdm/BHM_model_summary_v002_gain.csv
Saving to : /Users/pizarror/Library/CloudStorage/OneDrive-SharedLibraries-NationalInstitutesofHealth/NIMH CDN lab - Datasets/SDAN/utility/BHM/crdm/BHM_model_trace_v002_gain.pkl
Time to complete 16 aggregate BHM : 2.952729646364848 minutes
Saving split BHM : /Users/pizarror/Library/CloudStorage/OneDrive-SharedLibraries-NationalInstitutesofHealth/NIMH CDN lab - Datasets/SDAN/utility/BHM/split_CRDM_BHM_gain.csv


,alpha,beta,gamma,subject,domain
0,0.334099,-0.775077,3.611480,24546_adaptive,gain
1,0.571669,-0.071913,1.237807,24546_stable,gain
2,0.844859,0.003019,1.562373,24569_adaptive,gain
3,0.744953,0.054985,1.304188,24569_stable,gain
4,0.502580,0.068376,0.115481,24626_stable,gain
5,1.430205,0.190592,0.094499,24667_adaptive,gain
6,1.106883,-0.158505,0.173137,24667_stable,gain
7,0.589223,0.106420,0.229466,24731_stable,gain
8,0.330096,0.295919,0.418137,24734_stable,gain
9,0.311062,0.194878,3.493516,24740_adaptive,gain


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


Saving diagnostic plots to bh_dir : /Users/pizarror/Library/CloudStorage/OneDrive-SharedLibraries-NationalInstitutesofHealth/NIMH CDN lab - Datasets/SDAN/utility/24546_adaptive/crdm/bh


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


...

Saving diagnostic plots to bh_dir : /Users/pizarror/Library/CloudStorage/OneDrive-SharedLibraries-NationalInstitutesofHealth/NIMH CDN lab - Datasets/SDAN/utility/24751_stable/crdm/bh


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


Saving diagnostic plots to bh_dir : /Users/pizarror/Library/CloudStorage/OneDrive-SharedLibraries-NationalInstitutesofHealth/NIMH CDN lab - Datasets/SDAN/utility/24752_stable/crdm/bh


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


Saving diagnostic plots to bh_dir : /Users/pizarror/Library/CloudStorage/OneDrive-SharedLibraries-NationalInstitutesofHealth/NIMH CDN lab - Datasets/SDAN/utility/morkeh_stable/crdm/bh
